In [1]:
import re
import glob
import csv
from helpers import config
from helpers.loading import load_daily_data ,file_exist,get_all_dates
from helpers.algorithm import find_best_delay
import pandas as pd
import time
%load_ext autoreload
%autoreload 2

In [2]:
get_all_dates()

['2005-07-15',
 '2005-07-18',
 '2005-07-19',
 '2005-07-20',
 '2005-07-21',
 '2005-07-22',
 '2005-07-25',
 '2005-07-26',
 '2005-07-27',
 '2005-07-28',
 '2005-07-29',
 '2005-08-01',
 '2005-08-02',
 '2005-08-03',
 '2005-08-04',
 '2005-08-05',
 '2005-08-08',
 '2005-08-09',
 '2005-08-10',
 '2005-08-11',
 '2005-08-12',
 '2005-08-15',
 '2005-08-16',
 '2005-08-17',
 '2005-08-18',
 '2005-08-19',
 '2005-08-22',
 '2005-08-23',
 '2005-08-24',
 '2005-08-25',
 '2005-08-26',
 '2005-08-29',
 '2005-08-30',
 '2005-08-31',
 '2005-09-01',
 '2005-09-02',
 '2005-09-05',
 '2005-09-06',
 '2005-09-07',
 '2005-09-08',
 '2005-09-09',
 '2005-09-12',
 '2005-09-13',
 '2005-09-14',
 '2005-09-15',
 '2005-09-16',
 '2005-09-19',
 '2005-09-20',
 '2005-09-21',
 '2005-09-22',
 '2005-09-23',
 '2005-09-26',
 '2005-09-27',
 '2005-09-28',
 '2005-09-29',
 '2005-09-30',
 '2005-10-03',
 '2005-10-04',
 '2005-10-05',
 '2005-10-06',
 '2005-10-07',
 '2005-10-10',
 '2005-10-11',
 '2005-10-12',
 '2005-10-13',
 '2005-10-14',
 '2005-10-

In [4]:
all_dates = get_all_dates(config["stock"])
print(f"{len(all_dates)} dates to process")

3034 dates to process


In [5]:
fieldnames = ['date', 'market1','market2',"lag"]

results_path = config["files"]["results"]["all_best_lags"] # file where to write the computed lags

result_file_exists = file_exist(results_path) 
csvfile = open(results_path, 'a', newline='') 
writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

if result_file_exists:
    processed_dates = set(pd.read_csv(results_path).date.unique())
else: 
    # if the file is new, we need to write headers
    writer.writeheader()
    processed_dates = set()


max_iterations = 10
start_time = time.time()
date_count = 0 # number of dates processed
for date_id,date in enumerate(all_dates):
    print(f"date:{date}, {date_id}:{len(all_dates)}, {100*date_id/len(all_dates):0.3f}%", end="\r")
    try:
        daily_data = load_daily_data(date)
    except:
        # in case all markets do not provide data for the given date, we skip the date
        continue
    
    # we skip the current date if it has already been processed
    if date in processed_dates:
        continue
        
    for i,n1 in enumerate(daily_data):
        for j,n2 in enumerate(daily_data):
            if i>j: # avoid symetric (corr(a,b)=corr(b,a)) and meaningless (corr(a,a)=1) calculations
                best_delay, delays, correlations, los, his = find_best_delay(daily_data,n1,n2,step_size=1000)
                # write the computed result
                writer.writerow({'date': date, 'market1': n1,'market2': n2,'lag': best_delay})
                writer.writerow({'date': date, 'market1': n2,'market2': n1,'lag': -best_delay})
    csvfile.flush() # flush every time we processed a date
    date_count+=1
    if date_count>=max_iterations:
        break
print()
print(f"{date_count} dates processed in {time.time()-start_time:0.2f}s")
csvfile.close()

date:2017-04-12, 3033:3034, 99.967%
0 dates processed in 0.25s


In [6]:
daily_data = load_daily_data(all_dates[0])

TypeError: load_daily_data() missing 1 required positional argument: 'location'